In [1]:
import vastai
import asyncio
import random
import logging

logging.getLogger("Serverless").propagate = False
# or, if the library uses a package logger name:
logging.getLogger("vastai").propagate = False


In [ ]:
import asyncio
import random
import vastai

async def start_session():
    client = vastai.Serverless()
    session_a = None
    session_b = None

    try:
        endpoint = await client.get_endpoint("my-comfy-endpoint")

        payload = {
            "input": {
                "modifier": "Text2Image",
                "modifications": {
                    "prompt": "Generate a page from a peanuts comic strip.",
                    "width": 512,
                    "height": 512,
                    "steps": 10,
                    "seed": random.randint(1, 1000)
                }
            }
        }

        session_a = await endpoint.session()
        session_b = await endpoint.session()

        def print_response(response):
            print(response["response"]["output"][0]["local_path"])

        while True:
            await asyncio.gather(
                session_a.request("/generate/sync", payload).then(print_response),
                session_b.request("/generate/sync", payload).then(print_response),
            )

    except asyncio.CancelledError:
        print("Notebook cell cancelled.")
        raise

    finally:
        if session_a:
            await session_a.close()
        if session_b:
            await session_b.close()
        await client.close()
        print("Sessions closed.")


In [ ]:
await start_session()

In [ ]:
import asyncio
import random
import vastai
def status_printout(status):
    status_obj = status["response"]["status"]

    print(
        f"{status_obj['message']:<30} | "
        f"Train loss: {status_obj['train_loss']:>8.4f} | "
        f"Accuracy: {status_obj['val_acc'] * 100:>6.2f}%"
    )


async def start_training_session():
    async with vastai.Serverless() as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint-2")

        payload = {'max_train_batches_per_epoch': 100, "epochs" : 200}



        async with await endpoint.session() as session:
            async def on_close(session):
                await session.request(route="/cancel_task", payload={})
            session.set_on_close(on_close)
            await session.request(route="/start_task", payload=payload)
            count = 0
            while True:
                await asyncio.sleep(1)
                status = await session.request(route="/status", payload={})
                status_printout(status)
                if status["response"]["status"]["state"] != "running":
                    break
                if count > 15:
                    status = await session.request(route="/cancel_task", payload={})
                count += 1
        print("Training complete")

    print("Sessions closed.")


In [30]:
await asyncio.gather(start_training_session())

Training epoch 2/200 batch 40/100 | Train loss:   0.2025 | Accuracy:  90.97%
Training epoch 3/200 batch 100/100 | Train loss:   0.1240 | Accuracy:  95.16%
Training epoch 5/200 batch 40/100 | Train loss:   0.0951 | Accuracy:  96.56%
Training epoch 6/200 batch 100/100 | Train loss:   0.0836 | Accuracy:  97.03%
Training epoch 8/200 batch 50/100 | Train loss:   0.0810 | Accuracy:  97.12%
Training epoch 9/200 batch 100/100 | Train loss:   0.0614 | Accuracy:  97.62%
Training epoch 11/200 batch 60/100 | Train loss:   0.0569 | Accuracy:  97.62%
Validation epoch 12/200 complete | Train loss:   0.0451 | Accuracy:  97.59%
Training epoch 14/200 batch 80/100 | Train loss:   0.0406 | Accuracy:  97.81%
Training epoch 16/200 batch 10/100 | Train loss:   0.0596 | Accuracy:  97.81%
Training epoch 17/200 batch 100/100 | Train loss:   0.0465 | Accuracy:  98.00%
Training epoch 19/200 batch 40/100 | Train loss:   0.0386 | Accuracy:  98.47%
Training epoch 20/200 batch 100/100 | Train loss:   0.0296 | Accurac

[None]

In [31]:
await asyncio.gather(start_training_session(), start_training_session())

Training epoch 2/200 batch 40/100 | Train loss:   0.2025 | Accuracy:  90.97%
Training epoch 3/200 batch 100/100 | Train loss:   0.1239 | Accuracy:  95.12%
Training epoch 5/200 batch 60/100 | Train loss:   0.0940 | Accuracy:  96.62%
Training epoch 6/200 batch 100/100 | Train loss:   0.0830 | Accuracy:  97.03%
Training epoch 8/200 batch 90/100 | Train loss:   0.0770 | Accuracy:  97.12%
Training epoch 10/200 batch 20/100 | Train loss:   0.0502 | Accuracy:  97.72%
Training epoch 11/200 batch 100/100 | Train loss:   0.0556 | Accuracy:  97.72%
Training epoch 13/200 batch 70/100 | Train loss:   0.0465 | Accuracy:  97.47%
Training epoch 15/200 batch 10/100 | Train loss:   0.0418 | Accuracy:  97.56%
Training epoch 16/200 batch 100/100 | Train loss:   0.0507 | Accuracy:  97.75%
Training epoch 18/200 batch 30/100 | Train loss:   0.0462 | Accuracy:  97.84%
Training epoch 19/200 batch 100/100 | Train loss:   0.0372 | Accuracy:  98.53%
Training epoch 21/200 batch 50/100 | Train loss:   0.0316 | Accu

CancelledError: 

In [ ]:
import asyncio
import random
import vastai
def status_printout(status):
    status_obj = status["response"]["status"]

    print(
        f"{status_obj['message']:<30} | "
        f"Train loss: {status_obj['train_loss']:>8.4f} | "
        f"Accuracy: {status_obj['val_acc'] * 100:>6.2f}%"
    )


async def start_full_training_session():
    async with vastai.Serverless() as client:
        endpoint = await client.get_endpoint("my-pytorch-endpoint")





        session = await endpoint.session()
        payload = {'max_train_batches_per_epoch': 10, "epochs" : 20, "session_id": session.session_id}
        await session.request(route="/start_task", payload=payload)
        while True:
            await asyncio.sleep(1)
            status = await session.request(route="/status", payload={}, retry=False)
            print(status)
            #status_printout(status)
        print("Training complete")

    print("Sessions closed.")


In [14]:
await asyncio.gather(start_full_training_session())

{'response': {'ok': True, 'status': {'task_id': 'None', 'state': 'running', 'message': 'Training epoch 2/20 batch 10/10', 'created_at': 1766009902.8990679, 'started_at': 1766009902.8990684, 'finished_at': None, 'epoch': 2, 'step': 20, 'total_steps': 200, 'train_loss': 1.234742820262909, 'train_acc': 0.6984375, 'val_loss': 1.6601162767410278, 'val_acc': 0.5228125, 'last_update_at': 1766009903.8539338, 'config': {'epochs': 20, 'batch_size': 64, 'lr': 0.001, 'max_train_batches_per_epoch': 10, 'max_val_batches': 50, 'seed': 1337, 'data_dir': './data', 'num_workers': 2, 'device': 'auto', 'pin_memory': True, 'task_id': 'None'}, 'error_type': None, 'error': None}}, 'latency': 0.2139289379119873, 'url': 'https://199.68.217.31:24522', 'reuqest_idx': 0, 'auth_data': {'__request_id': 'ca74e7ff-7c71-45b5-9f70-f2f0aa2d1eb9', 'cost': 100.0, 'endpoint': 'my-pytorch-endpoint', 'reqnum': 0, 'request_idx': 6, 'signature': 'k2l6ztEGClqyzKL+H4othl7dob458J48cyWplFTVpY6P1E39FErvtI59sBlpZE8cwFF4eN07ysm46YZxH

CancelledError: 